In [113]:
from paper_wallet.wallet import PaperWallet
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import requests
import time
from keras.models import load_model

In [114]:
# --- Configurable Parameters ---
LIMIT_OFFSET_PERCENTAGE = 0.0005  # 0.05% offset for limit orders
BROKER_FEE_PERCENTAGE = 0.004     # 0.4% broker fee
TRADE_TRIGGER_THRESHOLD = 0.01    # Only trade if prediction > 1%

In [115]:
model = load_model('models/model.keras')
#Read test data
use_model_data = True
price_column = "Price"
df = pd.read_csv('model performance stats/prices.log', header=None, names=["Price", "Timestamp"])
if use_model_data:
    df = pd.read_csv('recent_eth_prices.csv')
    price_column = "price"

prices = np.array(df[price_column]).reshape(-1, 1)

In [116]:
#Set min-max scaler to same as model
model_data = pd.read_csv('resampled_data.csv')
model_prices = np.array(model_data[price_column])

# Load the same MinMaxScaler used during training
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = RobustScaler()
scaler.fit(model_prices.reshape(-1, 1))  # Fit on the current live prices

MinMaxScaler()

In [117]:
#read test data
df.head()

,Unnamed: 0,trade_id,product_id,price,size,time,side,bid,ask,exchange
0,0,631358351,ETH-USD,1555.87,0.061216,2025-04-08 00:44:45.530181+00:00,SELL,NaN,NaN,NaN
1,1,631359041,ETH-USD,1553.43,0.000007,2025-04-08 00:46:45.913575+00:00,BUY,NaN,NaN,NaN
2,2,631359591,ETH-USD,1553.84,0.004349,2025-04-08 00:48:45.733243+00:00,BUY,NaN,NaN,NaN
3,3,631360066,ETH-USD,1550.63,0.000348,2025-04-08 00:50:45.509438+00:00,BUY,NaN,NaN,NaN
4,4,631360501,ETH-USD,1553.09,0.007276,2025-04-08 00:52:45.804153+00:00,SELL,NaN,NaN,NaN


In [118]:
# Scale prices to match training format
prices_original = df[price_column].values.reshape(-1, 1)
df['transformed_price'] = scaler.transform(prices_original)
df['sma_5'] = df['transformed_price'].rolling(window=5).mean()
df['momentum'] = df['transformed_price'] - df['transformed_price'].shift(5)
df['price_scaled'] = scaler.fit_transform(df[[price_column]])
df['pct_change'] = df[price_column].pct_change() * 100  # Convert to percentage
df.dropna(subset=['sma_5', 'momentum'], inplace=True)
df.head()

prices = df['price_scaled'].values
sma_5 = df['sma_5'].values
momentum = df['momentum'].values

In [119]:
# Prepare the data for LSTM (same sequence length used in training)
X_test = []
last_prices = []

# Create sequences of data for LSTM
sequence_length = 120  # Sequence of 10 observations
forecast_steps = 60  # Predict 5 steps into the future

for i in range(0, len((prices) - sequence_length - forecast_steps), sequence_length):
    try:
        row = []
        row.append(np.array(prices[i:i + sequence_length]).reshape(sequence_length, 1))

        X_test.append(prices[i:i + sequence_length])
        last_prices.append(prices_original[i + sequence_length])
    except ValueError:
        break

X_test = np.array(X_test)

In [120]:
# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions to get actual price values
predicted_prices = scaler.inverse_transform(predictions)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


In [121]:
starting_value = 10000
wallet = PaperWallet(starting_usd=starting_value)

In [122]:
len(prices)

14309

In [124]:
#run through prices, make predictions, and modify paper wallet as such
buy_transactions = 0
sell_transactions = 0
threshold = 0.01 # don't take action unless prediction is at lest threshold % away


for i in range(len(predicted_prices)):
        #build feature set
        predicted_price_transformed = predicted_prices[i]
        last_price = last_prices[i]
        buy_eth = predicted_price_transformed > last_price

        #don't make trade unless prediction is big enough
        pct_change = (predicted_price_transformed - last_price)/last_price
        print(pct_change)
        if abs(pct_change) < threshold:
             continue

        #amount of wallet to trade on each transaction
        fraction = 0.2

        if buy_eth:
            usd_total = wallet.usd_balance
            trade_amount = usd_total * fraction

            wallet.buy_eth(trade_amount, last_price)
            buy_transactions += 1
        else:
            eth_total = wallet.eth_balance
            trade_amount = eth_total * fraction

            wallet.sell_eth(trade_amount, last_price)
            sell_transactions += 1

print(f"Buy Transactions: {buy_transactions} Sell Transactions: {sell_transactions}")
wallet.print_total_value(last_price)
print(f"ETH starting price: ${prices_original[0]}. Ending price: ${prices_original[-1]}")
print(f"ETH price change: {((prices_original[-1] - prices_original[0])/prices_original[0]) * 100}%")

Buy Transactions: 0 Sell Transactions: 0
Total Value: $[10000.]
Percent Difference: [0.]
ETH starting price: $[1555.87]. Ending price: $[1778.64]
ETH price change: [14.31803428]%
